In this Cell we drop 'url', 'urlid', 'IP' and 'region' and 'bidid' from the training data.

In [ ]:
import os
import csv
import numpy as np
import random
from random import shuffle
import math
import pandas as pd
from random import uniform
from sklearn.utils import resample
from sklearn.model_selection import train_test_split


filename = '../input/train.csv'

data = pd.read_csv(filename)


train = data.drop(['bidid',  'region',  'bidprice','url','urlid', 'IP'], axis=1)


train['slotvisibility'] = pd.to_numeric(train['slotvisibility'], errors='coerce')
avgSlot = train['slotvisibility'].mean()

vals = '../input/validation.csv'

valid = pd.read_csv(vals)


valid = valid.drop(['bidid',  'region',  'bidprice','url','urlid','IP'], axis=1)
valid['slotvisibility'] = pd.to_numeric(valid['slotvisibility'], errors='coerce')



fn = '../input/test.csv'

tests = pd.read_csv(fn)

tests = tests.drop(['region','url','urlid','IP'], axis=1)


small = train[train.click==1]
big = train[train.click==0]






In [ ]:
nsamp = int((len(big)*0.2))
#negatively downsample majority class at 0.2
nBig = resample(big, replace=False, n_samples=nsamp, random_state=123) 

Here we separate out clicks and payprice data from feature values

In [ ]:
train=nBig.append(small, ignore_index=True)


ids = tests['bidid']
tests = tests.drop(['bidid'], axis=1)
clicks = train['click']
train = train.drop(['click'], axis=1)
price = train['payprice']
train = train.drop(['payprice'], axis=1)



cVal = valid['click']
valid = valid.drop(['click'], axis=1)
vPrice = valid['payprice']
valid = valid.drop(['payprice'], axis=1)

We append validation and test data during feature engineering to ensure all data goes through the same tranformation

In [ ]:
train=train.append(valid, ignore_index=True)
train=train.append(tests, ignore_index=True)

here we iterate through all records and determine what usertags are present, recording for each tag the sample in which it occurs.

In [ ]:
from tqdm import tqdm
from sklearn import preprocessing
def getDict(train):
    
    tagD = {}

    for index, row in tqdm(train.iterrows()):
        for j in [x.strip() for x in row['usertag'].split(',')]:
            if j in tagD:
                tagD[j].append(index)
            else:
                tagD[j]=[index]
    
    
    return tagD




In [ ]:
train['usertag']=train['usertag'].fillna("")
tagD = getDict(train)

tagD.pop('', None)
    
    
for i in tagD:
    train[i]=0




We one-hot the usertags by accessing for each one the samples in which they occur

In [ ]:



for i in tqdm(tagD.keys()):
    #tags[i] = tags[i]
    
    train.iloc[tagD[i], train.columns.get_loc(i)] = 1

train = train.drop(['usertag'], axis=1)  

In [ ]:
from tqdm import tqdm
def transformDatas(df, avgSlot):
    
    df = df.replace("FourthView","4")
    df = df.replace("OtherView","0")
    df = df.replace("FirstView","1")
    df = df.replace("ThirdView","3")
    
    df['sin_time'] = np.sin(2*np.pi*df.hour/24) #encode circularity of 24 h clock
    df['cos_time'] = np.cos(2*np.pi*df.hour/24)
    df.drop('hour', axis=1, inplace=True)
    
    df['slotvisibility'] = pd.to_numeric(df['slotvisibility'], errors='coerce')
    df['slotvisibility'] = df['slotvisibility'].fillna(avgSlot)
    
    
    
    
    
            
    df = df.drop(['userid'],axis=1)
    
   # bins = pd.IntervalIndex.from_tuples([(-1, 0), (0, 10), (10, 50), (50,100),(100, float('inf'))])

   # df['slotprice']=pd.cut(train['slotprice'], bins)
    
    df['keypage'].fillna("", inplace=True)
    
    df['domain'].fillna("", inplace=True)
    
    df['adexchange'].fillna("", inplace=True)
    

    
    
    
    
   # df['slotprice']=df['slotprice'].cat.codes
    
    return df
    
    

In [ ]:

train = transformDatas(train, avgSlot)


In [ ]:
for f in train.columns: 
    if train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values)) 
        train[f] = lbl.transform(list(train[f].values))

In [ ]:
norms = ['slotwidth', 'slotheight', 'slotvisibility','slotprice', 'slotvisibility', 'sin_time', 'cos_time']

for feature_name in norms:
        max_value = train[feature_name].max()
        min_value = train[feature_name].min()
        train[feature_name] = (train[feature_name] - min_value) / (max_value - min_value)

In [ ]:
#categorical data

cats = ['weekday','useragent','city','adexchange' ,'domain' ,'slotid' , 'slotformat','creative','keypage','advertiser']

In [ ]:

    
train = pd.get_dummies(train,columns=cats, sparse =True)
tests = train.tail(len(tests))
train = train.head(len(train)-len(tests))

In [ ]:
valid = train.tail(len(valid))
train = train.head(len(train)-len(valid))

to prevent memory issues, we convert all dataframes to sparse.

In [ ]:
train = train.to_sparse(fill_value=None)

tests = tests.to_sparse(fill_value=None)

valid = valid.to_sparse(fill_value=None)

To then run the Logistic Regression, we then need to convert the matrix to sparse coordinate format

In [ ]:
train = train.to_coo()

tests = tests.to_coo()

valid = valid.to_coo()

In [ ]:
from sklearn.linear_model import LogisticRegression

regr = LogisticRegression()
Y= clicks

regr.fit(train, Y)
v_pred = regr.predict_proba(valid)

In [ ]:
#formula to correct for negative down-sampling
#v_pred = [x[1]/(x[1]+(1-(x[1]/0.2))) for x in v_pred]

In [ ]:
v_pred = [x[1] for x in v_pred]
v_clicks = cVal.values

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(v_clicks, v_pred)
avgCtr = 1793/(2429188+1793)

In [ ]:

true = pd.Series(clicks).values
price = pd.Series(price).values

In [ ]:
vTrue = pd.Series(cVal).values
vPrice = pd.Series(vPrice).values

In [ ]:
clist = []
highest_pay_price = max(price)+1

def clicksAtPrice(price, bid, y_pred, true):
  clicks = 0
  budget = 6250
  for i, p in enumerate(price):

    if budget >= p/1000 and bid*(y_pred[i]/avgCtr) >= p:
      budget -= p/1000
      clicks += true[i]

  return clicks

mClick = 0
bid = 0
for i in tqdm(range(0, 60)):
    
    a = clicksAtPrice(vPrice,i,v_pred,cVal)
    clist.append(a)
    
    if a>mClick:
        mClick=a
        bid = i
        
  

In [ ]:
bid_pred = regr.predict_proba(tests)

bid_pred = [x[1] for x in bid_pred]
biid = pd.Series(ids).values

output linear bid prices for the test set

In [ ]:
import csv

with open('linear1.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['bidid']  + ['bidprice']) 
    for i in range(0,len(biid)):
        writer.writerow([biid[i]]+ [bid*(bid_pred[i]/avgCtr)])